In [1]:
%load_ext autotime

time: 299 µs


In [2]:
import pandas as pd
import numpy as np
import _pickle as pickle
from datetime import datetime

time: 749 ms


In [3]:
claims_path = "/Users/xxz005/Desktop/Raw_Data/PFK_Jan 2009 - Jul 2017/Claims.txt"
Eligibility_path = "/Users/xxz005/Desktop/RAW_DATA/PFK_Jan 2009 - Jul 2017/Eligibility.txt"
Rxclaims_path = "/Users/xxz005/Desktop/Raw_Data/PFK_Jan 2009 - Jul 2017/Rxclaims.txt"

time: 488 µs


In [4]:
# x = pd.read_csv(claims_path,sep="|",nrows=10)
# for i, m in enumerate(x.columns):
#     print(i,m)

time: 346 µs


In [5]:
def process(claims_path, year_list):
    pid_svcdt_map = {}
    svcdt_info_map = {}
    pid_dob_map = {}
    pid_gender_map = {}
    
    with open(claims_path) as infile:
        next(infile)
        for i,line in enumerate(infile):
            line=line.split("|")
            pid = line[0]
            
            # information
            svcdt = line[27][:9]
            svcenddt = line[28][:9]
            if svcdt[-4:] not in year_list:continue
                        
            diag1 = line[13] #diag1~2 only 
            diag2 = line[14]
            proc = line[21]
            claimtype = line[71] 
            paid = line[46]
            
            svcdt_pid = svcdt+"||"+pid
            info = [diag1, diag2, proc, claimtype, paid, svcdt]                           
            
            if pid not in pid_svcdt_map:
                pid_svcdt_map[pid] = set([svcdt_pid])
            else:
                pid_svcdt_map[pid].add(svcdt_pid)
                
            if svcdt_pid not in svcdt_info_map:
                svcdt_info_map[svcdt_pid] = [info]
            else:
                svcdt_info_map[svcdt_pid].append(info)  
            
            # Demographic
            if len(line[51])<4:continue
            
            dob=line[51][:9]
            gender = line[10]
            
            if pid not in pid_dob_map: 
                pid_dob_map[pid] = [dob]
            else:
                pid_dob_map[pid].append(dob)
            
            if pid not in pid_gender_map: 
                pid_gender_map[pid] = [gender]
            else:
                pid_gender_map[pid].append(gender)
                
    return pid_svcdt_map, svcdt_info_map, pid_dob_map, pid_gender_map

time: 4.01 ms


In [6]:
pid_svcdt_map, svcdt_info_map, pid_dob_map, pid_gender_map = process(claims_path, ["2013", "2014"])

time: 5min 46s


In [7]:
def processRX(claims_path, year_list):
    pid_svcdt_map = {}
    svcdt_info_map = {}
    
    with open(claims_path) as infile:
        next(infile)
        for i,line in enumerate(infile):
            line=line.split("|")
            pid = line[0]
            
            # information
            svcdt = line[7][:9]
            if svcdt[-4:] not in year_list:continue
                
            Drugname = line[13]
            DrugClass = line[16]
            paid = line[19]
            
            svcdt_pid = svcdt+"||"+pid
            info = [svcdt, Drugname, DrugClass, paid]                                        
            
            if pid not in pid_svcdt_map:
                pid_svcdt_map[pid] = set([svcdt_pid])
            else:
                pid_svcdt_map[pid].add(svcdt_pid)
                
            if svcdt_pid not in svcdt_info_map:
                svcdt_info_map[svcdt_pid] = [info]
            else:
                svcdt_info_map[svcdt_pid].append(info)  
            
    return pid_svcdt_map, svcdt_info_map

time: 5.54 ms


In [8]:
pid_svcdt_map_RX, svcdt_info_map_RX = processRX(Rxclaims_path, ["2013", "2014"])

time: 2min 8s


In [9]:
def eli_extract(path):
    pid_eli_map = {}
    with open(path) as infile:
        next(infile)
        for line in infile:
            line = line.split('|')
            pid = line[0]
            eli_month = line[1]
            if pid not in pid_eli_map:
                pid_eli_map[pid] = [eli_month]
            else:
                pid_eli_map[pid].append(eli_month)
    return pid_eli_map

def check_eli(pid_eli_map, year_list):
    pid_list = []
    for p in pid_eli_map:
        eli_flag=1
        for year in year_list:
            months = [i for i in pid_eli_map[p] if i.startswith(year)]
            if len(months) <10:
                eli_flag=0
                break
        if eli_flag: pid_list.append(p)
    return pid_list

def age_flag(p, age, year):
    if p not in pid_dob_map:
        return 0
    else:
        return int(pid_dob_map[p][0][-4:]) < year-age

time: 10.5 ms


In [10]:
pid_eli_map = eli_extract(Eligibility_path)
pid_list = check_eli(pid_eli_map, ["2013", "2014"])
print("Total eligible patients:", len(pid_list))

pid_list = [p for p in pid_list if age_flag(p, 2, 2013)]
print("# of patients age>2:", len(pid_list))

Total eligible patients: 209978
# of patients age>2: 174093
time: 40.7 s


# Input data

In [11]:
StartDay = datetime.strptime('01JAN2013', '%d%b%Y')

pid_seq = []
age_seq = []
sex_seq = []
zip_seq = []

cost_seq_2013 = []
code_seq_2013 = []
util_seq_2013 = []
date_seq_2013 = []

cost_seq_2014 = []
code_seq_2014 = []
util_seq_2014 = []
date_seq_2014 = []

for i, pid in enumerate(pid_list):        
    code_2013 = []
    util_2013 = []
    cost_2013 = []
    date_2013 = []

    code_2014 = []
    util_2014 = []
    cost_2014 = []
    date_2014 = []
    
    for svcdt in pid_svcdt_map[pid]:
        svcdt_code = set()
        svcdt_type = set()
        svcdt_cost = 0
        
        for line_trans in svcdt_info_map[svcdt]:
            diag1, diag2, proc, claimtype, paid, svcdt = line_trans
            
            claim_date = (datetime.strptime(svcdt, '%d%b%Y') - StartDay).days
            paid = max(0, float(paid))
            
            if diag1: svcdt_code.add("D_"+diag1)
            if diag2: svcdt_code.add("D_"+diag2)
            if proc: svcdt_code.add("P_"+proc)
            svcdt_type.add(claimtype)
            svcdt_cost += max(0, float(paid))
        
        if svcdt[-4:]=="2013":
            code_2013.append(svcdt_code)
            util_2013.append(svcdt_type)
            cost_2013.append(svcdt_cost)
            date_2013.append(claim_date)
            
        elif svcdt[-4:]=="2014":
            code_2014.append(svcdt_code)
            util_2014.append(svcdt_type)
            cost_2014.append(svcdt_cost)
            date_2014.append(claim_date)
        
        else:
            print(pid)
    
    if pid in pid_svcdt_map_RX:
        for svcdt in pid_svcdt_map_RX[pid]:
            svcdt_code = set()
            svcdt_class = set()
            svcdt_type = set()
            svcdt_cost = 0

            for line_trans in svcdt_info_map_RX[svcdt]:
                svcdt, Drugname, DrugClass, paid = line_trans
                
                claim_date = (datetime.strptime(svcdt, '%d%b%Y') - StartDay).days
                claimtype = "RX"  
                
                if Drugname: svcdt_code.add("M_"+Drugname)
                if DrugClass: svcdt_class.add(DrugClass)
                svcdt_type.add(claimtype)
                svcdt_cost += max(0, float(paid))
            
            if svcdt[-4:]=="2013":
                code_2013.append(svcdt_code)
                util_2013.append(svcdt_type)
                cost_2013.append(svcdt_cost)
                date_2013.append(claim_date)
            
            elif svcdt[-4:]=="2014":
                code_2014.append(svcdt_code)
                util_2014.append(svcdt_type)
                cost_2014.append(svcdt_cost)
                date_2014.append(claim_date)
    
    if len(cost_2013)> 30:continue   # remove patient with visit times > k
    if sum(cost_2013)< 100: continue
        
    pid_seq.append(pid)
    
    sex_seq.append(pid_gender_map[pid][0])
    age_seq.append(2013 - int(pid_dob_map[pid][0][-4:]))    
    
    idx = np.argsort(date_2013)
    code_seq_2013.append(np.array(code_2013)[idx])
    cost_seq_2013.append(np.array(cost_2013)[idx])
    util_seq_2013.append(np.array(util_2013)[idx])
    date_seq_2013.append(np.array(date_2013)[idx])
    
    idx = np.argsort(date_2014)
    code_seq_2014.append(np.array(code_2014)[idx])
    cost_seq_2014.append(np.array(cost_2014)[idx])
    util_seq_2014.append(np.array(util_2014)[idx])
    date_seq_2014.append(np.array(date_2014)[idx])

time: 4min 30s


In [12]:
len(sex_seq)

144657

time: 25.4 ms


# Sanity Check

In [13]:
k=5

time: 578 µs


In [14]:
cost_seq_2013[k]

array([ 10.  , 151.28, 202.95,  47.41])

time: 23.8 ms


In [15]:
util_seq_2013[k]

array([{'PR'}, {'OP', 'PR'}, {'RX'}, {'RX'}], dtype=object)

time: 3.24 ms


In [16]:
date_seq_2013[k]

array([196, 293, 293, 294])

time: 11.2 ms


In [17]:
code_seq_2013[k]

array([{'P_90633', 'D_V05.3'},
       {'P_306', 'P_99214', 'D_788.1', 'D_493.00'},
       {'M_EASIVENT', 'M_VENTOLIN HFA', 'M_QVAR'}, {'M_CEFDINIR'}],
      dtype=object)

time: 5.88 ms


In [18]:
# del pid_svcdt_map, 
# del svcdt_info_map, 
# del pid_dob_map, 
# del pid_gender_map, 

time: 315 µs


In [19]:
pickle.dump(age_seq,open("age_seq","wb"))
pickle.dump(sex_seq,open("sex_seq","wb"))

pickle.dump(code_seq_2013,open("code_seq_2013","wb"))
pickle.dump(cost_seq_2013,open("cost_seq_2013","wb"))
pickle.dump(util_seq_2013,open("util_seq_2013","wb"))
pickle.dump(date_seq_2013,open("date_seq_2013","wb"))

pickle.dump(code_seq_2014,open("code_seq_2014","wb"))
pickle.dump(cost_seq_2014,open("cost_seq_2014","wb"))
pickle.dump(util_seq_2014,open("util_seq_2014","wb"))
pickle.dump(date_seq_2014,open("date_seq_2014","wb"))

time: 7min 5s


# process ccs

In [20]:
icd2cat = pickle.load(open("/Users/xxz005/Desktop/RAW_DATA/icd2ccs","rb"))
cpt2cat = pickle.load(open("/Users/xxz005/Desktop/RAW_DATA/cpt2ccs","rb"))
med2cat = pickle.load(open("/Users/xxz005/Desktop/RAW_DATA/drug2class","rb"))

time: 533 ms


In [21]:
diag2cat = {}
proc2cat = {}
drug2cat = {}

for p in code_seq_2013:
    for v in p:
        for c in v:
            if c[0] == "D":
                diag = c[2:].replace(".", "")
                if diag in icd2cat:
                    cat = icd2cat[diag]
                    diag2cat[c] = cat
            
            elif c[0] == "P":
                proc = c[2:]
                if proc in cpt2cat:
                    cat = cpt2cat[proc]
                    proc2cat[c] = cat
                    
            elif c[0] == "M":
                drug = c[2:]
                if drug in med2cat:
                    cat = med2cat[drug]
                    drug2cat[c] = cat
            else:
                print(c)

for p in code_seq_2014:
    for v in p:
        for c in v:
            if c[0] == "D":
                diag = c[2:].replace(".", "")
                if diag in icd2cat:
                    cat = icd2cat[diag]
                    diag2cat[c] = cat
            
            elif c[0] == "P":
                proc = c[2:]
                if proc in cpt2cat:
                    cat = cpt2cat[proc]
                    proc2cat[c] = cat
                    
            elif c[0] == "M":
                drug = c[2:]
                if drug in med2cat:
                    cat = med2cat[drug]
                    drug2cat[c] = cat
            else:
                print(c)

time: 14.2 s


In [22]:
len(diag2cat), len(proc2cat), len(drug2cat)

(6416, 4265, 2643)

time: 12.7 ms


In [23]:
vocab = list(set([x for p in code_seq_2013 for v in p for x in v]))

len([x for x in vocab if x[0]=="D"]), len([x for x in vocab if x[0]=="P"]), len([x for x in vocab if x[0]=="M"])

(6321, 3659, 2113)

time: 1.43 s


In [24]:
pickle.dump(diag2cat,open("CCS/diag2cat","wb"))
pickle.dump(proc2cat,open("CCS/proc2cat","wb"))
pickle.dump(drug2cat,open("CCS/drug2cat","wb"))

time: 12.8 ms
